# Sign In with LinkedIn API

Most APIs from Linkedin use a 3-legged authorization code, which means that it's not enough for the app to authenticate with the server, the member whose data is being collected has to give their authorization as well.

Client credential flow, commonly referred to as "OAuth two-legged", is lock out by default. With this flow the app would be able to authorize with LinkedIn's API directly - outside the context of any specific user. Permission to use this flow is only grantedf by direct request.

With the "Sign In with LinkedIn" API we can get access to the user's profile information.

1. Go to https://www.linkedin.com/developers/apps and create an app
2. Add the authentication keys (Client ID + Client Secret) to a `.env` file as  `CLIENT_ID` and `CLIENT_SECRET` respectively
3. Add a redirect URL
4. Assign the redirect URL you've added to your `.env` file as `REDIRECT_URL`

In [ ]:
import os
import string
import random
from pathlib import Path

import requests
from dotenv import load_dotenv

load_dotenv()

CLIENT_ID = os.getenv("CLIENT_ID")
CLIENT_SECRET = os.getenv("CLIENT_SECRET")
REDIRECT_URL = os.getenv("REDIRECT_URL")

AUTH_URL = "https://www.linkedin.com/oauth/v2/authorization"
ACCESS_TOKEN_URL = "https://www.linkedin.com/oauth/v2/accessToken"
ME_URL = "https://www.linkedin.com/oauth/v2/me"

In [ ]:
def _generate_unique_string() -> str:
    vocab = string.ascii_letters
    string_length = 24
    return ''.join(random.choice(vocab) for _ in range(string_length))

def get_auth_link() -> None:
    scope = "r_liteprofile,r_emailaddress"
    param = {
        "response_type": "code",
        "client_id": CLIENT_ID,
        "redirect_uri": REDIRECT_URL,
        "scope": scope,
        # state is a unique string value of your choice that is hard to guess.
        # Used to prevent CSRF
        "state": _generate_unique_string()
    }
    response = requests.get(url=AUTH_URL, params=param)
    return_url = response.url
    print("Please click the URL and authenticate in order to get your auth code")
    print("")
    print(return_url)


get_auth_link()


After giving permissions, your browser will open the redirect URL you've provided. The authentication token is added as a URL param of that URL. Go to your browser address bar, copy is and add it to your `.env` file as `TOKEN`.

In [ ]:
load_dotenv()

AUTH_CODE = os.getenv("AUTH_CODE")

The next step is to get an access token for your application using the authorization code from the previous step.

In [ ]:
headers = {
    "Content-Type": "application/x-www-form-urlencoded",
}
data = {
    "grant_type": "authorization_code",
    "code": AUTH_CODE,
    "client_id": CLIENT_ID,
    "client_secret": CLIENT_SECRET,
    "redirect_uri": REDIRECT_URL,
}
response = requests.post(ACCESS_TOKEN_URL, data=data)
data = response.json()
access_token = data.get("access_token")

if access_token:
    env_path = Path().cwd().parent / ".env"
    with open(env_path, "a") as file:
        file.write(f"ACCESS_TOKEN = {access_token}")


In [ ]:
load_dotenv()

ACCESS_TOKEN = os.getenv("ACCESS_TOKEN")

## Personal profile

In [ ]:
import json

params = {"oauth2_access_token": ACCESS_TOKEN}
response = requests.get("https://api.linkedin.com/v2/me", params=params)

print(json.dumps(response.json(), indent=1))

### Displaying specific fields

In [ ]:
params = {
    "oauth2_access_token": ACCESS_TOKEN,
    "fields": ["localizedFirstName,localizedLastName,id"],
}
response = requests.get("https://api.linkedin.com/v2/me", params=params)

print(json.dumps(response.json(), indent=1))

In [ ]:
params = {
    "oauth2_access_token": ACCESS_TOKEN,
    "fields": ["lastName:(preferredLocale:(country,language))"],
}
response = requests.get("https://api.linkedin.com/v2/me", params=params)

print(json.dumps(response.json(), indent=1))